In [1]:
import MsgPack
using ZMQ
import ZMQ: send

In [2]:
type LazyPirate
    ctx::Context
    socket::Socket
    server_endpoint::String
end

function send(pirate::LazyPirate, msg)
    while true
        send(pirate.socket, Message(msg))
        
        c = Channel()
        @async try
            put!(c, (ZMQ.recv(pirate.socket), :received))
        catch e
            put!(c, (e, :errored))
        end
        @async begin
            sleep(1)
            put!(c, (nothing, :timedout))
        end
        data, status = take!(c)
        
        if status == :errored
            rethrow(data)
        elseif status == :timedout
            ZMQ.set_linger(pirate.socket, 0)
            ZMQ.close(pirate.socket)
            pirate.socket = Socket(pirate.ctx, REQ)
            ZMQ.connect(pirate.socket, pirate.server_endpoint)   
        else  # status == :received
            break
        end
    end
end
    

send (generic function with 10 methods)

In [5]:
ctx = Context()
s = Socket(ctx, REQ)
ZMQ.connect(s, "tcp://localhost:5555")

In [6]:
ZMQ.send(s, Message("Hello"))

In [5]:
pirate = LazyPirate(ctx, s, "tcp://localhost:5555")

LazyPirate(ZMQ.Context(Ptr{Void} @0x00000000024d7eb0,ZMQ.Socket[ZMQ.Socket(Ptr{Void} @0x0000000003e2e780,Base.Filesystem._FDWatcher(Ptr{Void} @0x0000000002b03f10,(1,0),Condition(Any[]),(false,false)))]),ZMQ.Socket(Ptr{Void} @0x0000000003e2e780,Base.Filesystem._FDWatcher(Ptr{Void} @0x0000000002b03f10,(1,0),Condition(Any[]),(false,false))),"tcp://localhost:5555")

In [ ]:
send(pirate, MsgPack.pack("hello world"))